In [14]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()

import pystan

In [17]:
log_dir = 'log/fabian_sim/20210824_183816_sim_m7/'
data = load_obj('data', log_dir)


## Run MCMC

In [18]:

model_num = 7

sm = load_obj('sm', 'log/compiled_models/model'+str(model_num)+'/')

param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
stan_names = model_phonebook(model_num)['stan_names']

fit_run = run_mcmc(
    data.get_stan_data(),
    sm,
    num_samples = 1000,
    num_warmup = 1000,
    adapt_engaged=True,
    num_chains = 1,
    log_dir = './log/20210725_224158_debug/'
)
ps = fit_run.extract(permuted=False, pars=param_names)


## Run IBIS

In [19]:
# load existing results

ibis = load_obj('particles', log_dir)


In [20]:
gen_model = 0
model_num = 1


for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(ibis.particles[name])
    w = exp_and_normalise(ibis.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))



Estimate
[-0.02  0.01 -0.07 -0.02  0.02  0.02]


Estimate
[[1.01 0.53 0.43 0.21 0.2  0.22]
 [0.53 0.97 0.43 0.22 0.2  0.22]
 [0.43 0.43 0.89 0.17 0.16 0.18]
 [0.21 0.22 0.17 0.97 0.5  0.54]
 [0.2  0.2  0.16 0.5  0.94 0.51]
 [0.22 0.22 0.18 0.54 0.51 0.9 ]]


## Plot Both results

In [21]:
# have to resample particles to get rid of weights
ibis.resample_particles()
particles = ibis.particles


In [22]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'mcmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))

alt.Chart(...)

In [23]:
param = 'alpha'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'hmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))


alt.Chart(...)